In [ ]:
project = 'derivative_project01'
import matplotlib
#matplotlib.use('Qt5Agg')
import matplotlib.pyplot as plt
import os
import mne
from mne import io
from mne.preprocessing import (ICA, create_eog_epochs, create_ecg_epochs, corrmap)
import zipfile
import sys
import pandas as pd
import numpy as np


In [4]:
# Subjects
sub_final = sorted(os.listdir('./data'))
print('    > subjects:', len (sub_final))
# Files
fol_names = []
for i in sub_final:
    subfols = os.listdir(f'./data/{i}')
    for s in subfols:
        fol_names.append(s)
print('    > files:',len(fol_names))

    > subjects: 70
    > files: 169


# Crop, Downsample, & Save New Files


In [5]:
#### Example using a Subset of 13 patients ###
##############################################
sub_final=sorted(['sub-CCTC07NETICU','sub-NETICU31','sub-NETICU33','sub-NETICU38','sub-NETICU40','sub-NETICU41','sub-NETICU46','sub-CCTC07','sub-MSICU32','sub-MSICU38','sub-MSICU42','sub-MSICU43','sub-MSICU47'])
fol_names = []
for i in sub_final:
    subfols = os.listdir(f'./data/{i}')
    for s in subfols:
        fol_names.append(s)
print('    > files:',len(fol_names))

13
    > files: 37


In [10]:
matplotlib.use('Qt5Agg')

#####################
### Cropping data ###
#####################

var_names = []
print ("####################################################################")
print ("### 'no' to skip this event; 'exit' to stop code; 'plot' to plot ###")
print ("### ; name+'_smin' to set dif smin time; 'zoom' for second plot  ###")
print ("####################################################################")
#for f in ['sub-CCTC03_task-taken_eeg','sub-CCTC03_task-rest02_eeg']:
for f in fol_names:
# for f in ['sub-HC05_task-Taken_eeg']:
    print ('\n______________________',f,'______________________')
    ID = ((f[4:]).split("_task"))[0] # f[4:] removes 'sub-' from f, Then the .split() will split it every "_task", Then [0] picks the ID "CCTC03"
    source_path = f'./data/sub-{ID}/{f}'
    fname = (sorted(os.listdir(source_path)))[0]
    data_path = f'{source_path}/{fname}'
    # Loading data
    with io.capture_output() as captured:
        raw = mne.io.read_raw_egi(data_path, preload=True)
    data = raw.copy()
    raw_sfreq = raw.info['sfreq']

    #Timing############################################################################################
    meas_data = data.info['meas_date']
    print(f'\n    - {fname}',
          f'\n    - {meas_data}',
          f'\n    - {data}',
          f'\n    - time(s): {(data.last_samp+1)/raw_sfreq}',
          f'\n    - first_samp-last_samp: {data.first_samp} - {data.last_samp}')
    seconds = (data.last_samp+1)/raw_sfreq
    seconds = seconds % (24 * 3600)
    hour = seconds // 3600
    seconds %= 3600
    minutes = seconds // 60
    seconds %= 60
    print("    - Duration = %dhr : %02dmin : %02dsec" % (hour, minutes, seconds))
    #EVENTS############################################################################################
    triggers = []
    for i in data.ch_names:
        if not 'E' in i:
            if not 'sync' in i:
                if not 'STI' in i:
                    if not 'V' in i:
                        print (f"    - Trigger <  ch. {i}>")
                        triggers.append(i)
    with io.capture_output() as captured:
        events = mne.find_events(data, stim_channel='STI 014', initial_event=True)
    for i in range(events.shape[0]):
        vars()['event'+str(i+1)] = events[i,0]
        print (f'{i+1} - {events[i,0]} / {raw_sfreq} = {(events[i,0])/raw_sfreq}')

    #crop####################################################################################################
    for i in range(events.shape[0]):
        name = input(f'Name event{i+1} for {f}:')
        if name == 'plot':
            with io.capture_output() as captured:
                data.plot(duration=200,
                          n_channels=50,
                          scalings=1000e-6, #20e-6`` (20µV)
                          #show_options=True,
                          #group_by='position',
                          butterfly=False,
                          title=f,
                          clipping=None,
                          block=True)
            time.sleep(2)
            name = input(f'Name event{i+1} for {f}:')
        if name == 'exit':
            print ('\n\n________________________________________________________________________________\n________________________________________________________________________________\n    Saved Variables:')
            for var in var_names:
                print(f'    --> {var}')
            print (f'    Checkpoint < event{i+1} for {f}\n________________________________________________________________________________\n________________________________________________________________________________\n')
            sys.exit()
        if 'no' not in name:
            if '_smin' in name:
                name = (name[:-5])
                smin = input(f'> smin to replace event{i+1} start time (type "zoom" to plot a closer look):')
                if smin == 'zoom':
                    with io.capture_output() as captured:
                        data.plot(duration=10,
                                  n_channels=10,
                                  scalings=1000e-6, #20e-6`` (20µV)
                                  #show_options=True,
                                  #group_by='position',
                                  butterfly=False,
                                  title=f,
                                  clipping=None,
                                  block=True)
                    time.sleep(2)
                    smin = int(input(f'> smin to replace event{i+1} start time:'))
                else:
                    smin = int(smin)
                vars()[ID+name+'_______________'+f]= nk.mne_crop(data.copy(),smin=smin, smax=smin+(312*raw_sfreq), include_tmax=False)
                var_names.append(f'{ID}{name}_______________{f}')
            else:
                vars()[ID+name+'_______________'+f]= nk.mne_crop(data.copy(),smin=vars()['event'+str(i+1)], smax=vars()['event'+str(i+1)]+(312*raw_sfreq), include_tmax=False)
                var_names.append(f'{ID}{name}_______________{f}')# 15x(_______________)
        if '_RS1st' in name:
            nameid = (name.split('RS1st'))[0]
            smin = int(input(f'> smin for RS2nd in {f}:'))
            vars()[ID+nameid+'RS2nd_______________'+f]= nk.mne_crop(data.copy(),smin=smin, smax=smin+(312*raw_sfreq), include_tmax=False)
            var_names.append(f'{ID}{nameid}RS2nd_______________{f}')
    #if no triggers
    if len(events) == 0 and len(triggers) == 0:
        name = input(f'Name file for {f}:')
        if name == 'plot':
            with io.capture_output() as captured:
                data.plot(duration=200,
                          n_channels=50,
                          scalings=1000e-6, #20e-6`` (20µV)
                          #show_options=True,
                          #group_by='position',
                          butterfly=False,
                          title=f,
                          clipping=None,
                          block=True)
            time.sleep(2)
            name = input(f'Name for file {f}:')
        if name == 'exit':
            print ('\n\n________________________________________________________________________________\n________________________________________________________________________________\n    Saved Variables:')
            for var in var_names:
                print(f'    --> {var}')
            print (f'    Checkpoint < for {f} (no events)\n________________________________________________________________________________\n________________________________________________________________________________\n')
            sys.exit()
        if 'no' not in name:
            missingtrig_smin = int(input(f'> smin value (samplingrate={raw_sfreq}) for {f}:'))
            vars()[ID+name+'_______________'+f]= nk.mne_crop(data.copy(),smin=missingtrig_smin, smax=missingtrig_smin+(312*raw_sfreq), include_tmax=False)
            var_names.append(f'{ID}{name}_______________{f}')
        if '_RS1st' in name:
            nameid = (name.split('RS1st'))[0]
            smin = int(input(f'> smin for RS2nd in {f}:'))
            vars()[ID+nameid+'RS2nd_______________'+f]= nk.mne_crop(data.copy(),smin=smin, smax=smin+(312*raw_sfreq), include_tmax=False)
            var_names.append(f'{ID}{nameid}RS2nd_______________{f}')
var_names



####################################################################
### 'no' to skip this event; 'exit' to stop code; 'plot' to plot ###
### ; name+'_smin' to set dif smin time; 'zoom' for second plot  ###
####################################################################

______________________ sub-CCTC07_task-rest_eeg ______________________

    - Session 20240325 1414.mff 
    - 2024-03-25 19:25:43.762000+00:00 
    - <RawMff | signal1.bin, 133 x 310630 (310.6 s), ~315.4 MB, data loaded> 
    - time(s): 310.63 
    - first_samp-last_samp: 0 - 310629
    - Duration = 0hr : 05min : 10sec
    - Trigger <  ch. 09  >
    - Trigger <  ch. 19  >
1 - 5051 / 1000.0 = 5.051
2 - 305139 / 1000.0 = 305.139
Name event1 for sub-CCTC07_task-rest_eeg:_RS
Name event2 for sub-CCTC07_task-rest_eeg:no

______________________ sub-CCTC07_task-taken_eeg ______________________

    - Session 20240325 1431.mff 
    - 2024-03-25 19:31:35.663000+00:00 
    - <RawMff | signal1.bin, 135 x 643354 (643.4 s), ~6


    - Session 20240417 1005.mff 
    - 2024-04-17 15:13:20.578000+00:00 
    - <RawMff | signal1.bin, 133 x 360475 (360.5 s), ~366.0 MB, data loaded> 
    - time(s): 360.475 
    - first_samp-last_samp: 0 - 360474
    - Duration = 0hr : 06min : 00sec
    - Trigger <  ch. 09  >
    - Trigger <  ch. 19  >
1 - 5070 / 1000.0 = 5.07
2 - 355144 / 1000.0 = 355.144
Name event1 for sub-MSICU43_task-rest_eeg:_RS
Name event2 for sub-MSICU43_task-rest_eeg:no

______________________ sub-MSICU43_task-taken_eeg ______________________

    - Session 20240417 1019.mff 
    - 2024-04-17 15:21:21.902000+00:00 
    - <RawMff | signal1.bin, 135 x 643505 (643.5 s), ~663.0 MB, data loaded> 
    - time(s): 643.505 
    - first_samp-last_samp: 0 - 643504
    - Duration = 0hr : 10min : 43sec
    - Trigger <  ch. 22  >
    - Trigger <  ch. 32  >
    - Trigger <  ch. 20  >
    - Trigger <  ch. 30  >
1 - 7740 / 1000.0 = 7.74
2 - 310591 / 1000.0 = 310.591
3 - 320742 / 1000.0 = 320.742
4 - 632936 / 1000.0 = 632.936

Name event1 for sub-NETICU40_task-sedoffrest_eeg:sedoff_RS
Name event2 for sub-NETICU40_task-sedoffrest_eeg:no

______________________ sub-NETICU40_task-sedon1taken_eeg ______________________

    - Session 20240403 0733.mff 
    - 2024-04-03 12:33:33.810000+00:00 
    - <RawMff | signal1.bin, 135 x 643706 (643.7 s), ~663.2 MB, data loaded> 
    - time(s): 643.706 
    - first_samp-last_samp: 0 - 643705
    - Duration = 0hr : 10min : 43sec
    - Trigger <  ch. 22  >
    - Trigger <  ch. 32  >
    - Trigger <  ch. 20  >
    - Trigger <  ch. 30  >
1 - 8121 / 1000.0 = 8.121
2 - 310800 / 1000.0 = 310.8
3 - 320955 / 1000.0 = 320.955
4 - 633149 / 1000.0 = 633.149
Name event1 for sub-NETICU40_task-sedon1taken_eeg:sedon1_ST
Name event2 for sub-NETICU40_task-sedon1taken_eeg:no
Name event3 for sub-NETICU40_task-sedon1taken_eeg:sedon1_IT
Name event4 for sub-NETICU40_task-sedon1taken_eeg:no

______________________ sub-NETICU40_task-sedon1rest_eeg ______________________

    - Session 20240403 0744

['CCTC07_RS_______________sub-CCTC07_task-rest_eeg',
 'CCTC07_ST_______________sub-CCTC07_task-taken_eeg',
 'CCTC07_IT_______________sub-CCTC07_task-taken_eeg',
 'CCTC07NETICUsedon1_RS_______________sub-CCTC07NETICU_task-sedon1rest_eeg',
 'CCTC07NETICUsedoff_RS_______________sub-CCTC07NETICU_task-sedoffrest_eeg',
 'CCTC07NETICUsedoff_ST_______________sub-CCTC07NETICU_task-sedofftaken_eeg',
 'CCTC07NETICUsedoff_IT_______________sub-CCTC07NETICU_task-sedofftaken_eeg',
 'CCTC07NETICUsedon1_ST_______________sub-CCTC07NETICU_task-sedon1taken_eeg',
 'CCTC07NETICUsedon1_IT_______________sub-CCTC07NETICU_task-sedon1taken_eeg',
 'MSICU32_RS_______________sub-MSICU32_task-rest_eeg',
 'MSICU32_ST_______________sub-MSICU32_task-taken_eeg',
 'MSICU32_IT_______________sub-MSICU32_task-taken_eeg',
 'MSICU38_ST_______________sub-MSICU38_task-taken_eeg',
 'MSICU38_IT_______________sub-MSICU38_task-taken_eeg',
 'MSICU38_RS_______________sub-MSICU38_task-rest_eeg',
 'MSICU42_RS_______________sub-MSICU42_

In [21]:
var_names.remove('MSICU47np_______________sub-MSICU47_task-taken_eeg')

## Downsample

In [ ]:
# Downsampling to 250Hz & renamed
downsampled_names=[]
for Vname in var_names:
    print (Vname)
    vars()[(Vname.split('_______________'))[0]] = vars()[Vname].copy()
    downsampled_names.append((Vname.split('_______________'))[0])
    SRate= vars()[(Vname.split('_______________'))[0]].info['sfreq']
    if SRate > 250:
        with io.capture_output() as captured:
            vars()[(Vname.split('_______________'))[0]].resample(250) 
        #print(f'    > {Vname} Data was downsampled to 250Hz from {raw_sfreq}')
    if SRate < 250: 
        print(f'    > Error: Data sampling frequency is {SRate}Hz.')
        answer = input('      - Do you want to continue(yes/no)?:')
        if answer.lower().startswith("y"):
            print("      - ok, carry on then")
        elif answer.lower().startswith("n"):
            print("      - ok, sayonnara")
            sys.exit()

# if raw.info['sfreq'] != 250:
#     raw = raw.resample(250)

## Save Individual Files 

In [ ]:
#make cutdata folder
if os.path.isdir('./cutdata')==False:
    os.makedirs('./cutdata')
#save
for V in downsampled_names:
    path = f'./cutdata/{V}'
    if not os.path.exists(path):
        os.makedirs(path)
        print (f'{path} path created')
    with io.capture_output() as captured:
        vars()[V].save(f'{path}/{V}.fif', overwrite=True)
        #vars()[V].save(f'{path}/{V}_eeg.fif', overwrite=True)

## Load .fif Files

In [29]:
fnames = sorted(os.listdir('./cutdata'))
for i in fnames:
    with io.capture_output() as captured:
        raw = mne.io.read_raw_fif(f'./cutdata/{i}/{i}.fif', preload=True)
    print (i, raw.info['sfreq'])
    vars()[i] = raw

CCTC03_IT 250.0
CCTC03_RS 250.0
CCTC03_ST 250.0
CCTC03part2_IT 250.0
CCTC03part2_RS 250.0
CCTC03part2_ST 250.0
CCTC04_IT 250.0
CCTC04_RS 250.0
CCTC04_RSredo 250.0
CCTC04_ST 250.0
CCTC05_IT 250.0
CCTC05_RS 250.0
CCTC05_ST 250.0
CCTC07NETICUsedoff_IT 250.0
CCTC07NETICUsedoff_RS 250.0
CCTC07NETICUsedoff_ST 250.0
CCTC07NETICUsedon1_IT 250.0
CCTC07NETICUsedon1_RS 250.0
CCTC07NETICUsedon1_ST 250.0
CCTC07_IT 250.0
CCTC07_RS 250.0
CCTC07_ST 250.0
HC01_IT 250.0
HC01_ST 250.0
HC02_IT 250.0
HC02_ST 250.0
HC03_IT 250.0
HC03_ST 250.0
HC04_IT 250.0
HC04_ST 250.0
HC05_IT1st 250.0
HC05_IT2nd 250.0
HC05_ST 250.0
HC06_IT 250.0
HC06_ST 250.0
HC07_IT 250.0
HC07_ST 250.0
HC08_IT 250.0
HC08_ST 250.0
HC09_IT 250.0
HC09_ST 250.0
HC10_IT 250.0
HC10_ST 250.0
HC11_IT 250.0
HC11_ST 250.0
HC12_IT 250.0
HC12_ST 250.0
HC13_IT 250.0
HC13_ST 250.0
HC14_IT 250.0
HC14_ST 250.0
HC15_IT 250.0
HC15_ST 250.0
HC16_IT 250.0
HC16_ST 250.0
HC17_IT 250.0
HC17_ST 250.0
HC18_IT 250.0
HC18_ST 250.0
MSICU05_IT 250.0
MSICU05_RS 250.0